In [ ]:
from keras.models import Model
from keras.layers import Input,Conv2D, Conv3D, MaxPooling3D,UpSampling2D, UpSampling3D,MaxPooling2D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate,Add
import keras as Ks
from keras.layers import Input
from glob import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2
from skimage import io
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=9201dfc3deaf62dd7c2043647ce5a1e3fd720bbd5571ca0c6e0328c8a61a3ca3
  Stored in directory: /tmp/pip-ephem-wheel-cache-oox6g631/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ
To: /content/CT_Data.zip
100% 624M/624M [00:03<00:00, 197MB/s]


In [ ]:
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ
To: /content/CT_Data.zip
100% 624M/624M [00:03<00:00, 194MB/s]


In [ ]:
from zipfile import ZipFile

file_name="/content/CT_Data.zip"

with ZipFile(file_name, 'r') as ziip:
  ziip.extractall()
  print('woow')

woow


In [ ]:
mask_files = glob('/content/content/drive/MyDrive/CT_Data/Masks/*')
len(mask_files)

7367

In [ ]:
CT = glob('/content/content/drive/MyDrive/CT_Data/CT_images/*')
len(CT)

7367

In [ ]:
df = pd.DataFrame({"image_path": CT, "mask_path":mask_files})


In [ ]:
Xtrain, Xval = train_test_split(df, test_size=0.15)
Xtest, Xval = train_test_split(Xval, test_size=0.5)
print("Train size is {}, valid size is {} & test size is {}".format(len(Xtrain), len(Xval), len(Xtest)))

Train size is 6261, valid size is 553 & test size is 553


In [ ]:
train_ids = list(Xtrain.image_path)
train_mask = list(Xtrain.mask_path)

val_ids = list(Xval.image_path)
val_mask= list(Xval.mask_path)

In [ ]:
def tversky(ytrue, ypred):
    ypredpos = K.flatten(ypred)
    ytruepos = K.flatten(ytrue)
    truepos = K.sum(ytruepos * ypredpos)
    falseneg = K.sum(ytruepos * (1-ypredpos))
    falsepos = K.sum((1-ytruepos)*ypredpos)
    alpha = 0.7
    smooth=100
    return (truepos + smooth)/(truepos + alpha*falseneg + (1-alpha)*falsepos + smooth)

def focaltversky(ytrue,ypred):
    ypred = tf.cast(ypred, tf.float32)
    ytrue = tf.cast(ytrue, tf.float32)
    
    pt_1 = tversky(ytrue, ypred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

def tversky_loss(ytrue, ypred):
    return 1 - tversky(ytrue,ypred)

In [ ]:
def resblock(X, f):
 
    Xcopy = X  
    
    # main path
    X = Conv2D(f, kernel_size=(1,1), kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(f, kernel_size=(3,3), padding='same', kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    
    # shortcut path
    Xcopy = Conv2D(f, kernel_size=(1,1), kernel_initializer='he_normal')(Xcopy)
    Xcopy = BatchNormalization()(Xcopy)
    
    # Adding the output from main path and short path together
    X = Add()([X, Xcopy])
    X = Activation('relu')(X)
    
    return X

def attention_block(L,X ,numfilters):
    ## Getting shapes if the layers
    shape_X=X.shape
    shape_L=L.shape

    ## getting x to the same shape as L
    theta_x=Conv2D(numfilters,(1,1),strides=(1,1),padding="same")(X)
    shape_X_theta=theta_x.shape
    phy_L=Conv2D(numfilters,(1,1),strides=(2,2),padding="same")(L)
    shape_L_phy=phy_L.shape
    ##Adding the 2 layers together
    XL= Ks.layers.add([theta_x,phy_L])
    activate_XL=Ks.layers.Activation('relu')(XL)
    conv=Conv2D(1,(1,1),padding='same')(activate_XL)
    
    act_sigmoid=Ks.layers.Activation('sigmoid')(conv)
    shape_sig=act_sigmoid.shape
    ##upsampling 
    upsampled=Ks.layers.UpSampling2D(size=(shape_L[1]//shape_sig[1],shape_L[2]//shape_sig[2]))(act_sigmoid)

    #output
    y=Ks.layers.multiply([upsampled,L])
    out=Conv2D(shape_L[3],(1,1),padding='same')(y)
    return out


def upsample_concat_with_attention_block(x, skip,num_filters):
    '''
    funtion for upsampling image
    '''
    m=attention_block(skip,x,num_filters)
    S= UpSampling2D((2,2))(x)
    # X= Concatenate()([S,skip])
    
    merge = Concatenate()([m,S])
    
    return merge

In [ ]:
def Res_Unet_with_att_model(inputshape=(256,256,3),filter_num=8,pool_size=(2, 2),n_classes=3):
  Xinput = Input(inputshape)

  conv_1 = Conv2D(2*filter_num, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(Xinput)
  conv_1 = BatchNormalization()(conv_1)
  conv_1 = Conv2D(2*filter_num, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv_1)
  conv_1 = BatchNormalization()(conv_1)
  pool_1 = MaxPool2D(pool_size)(conv_1)


  conv_2 = resblock(pool_1, 4*filter_num)
  pool_2 = MaxPooling2D(pool_size)(conv_2)


  conv_3 = resblock(pool_2, 8*filter_num)
  pool_3 = MaxPooling2D(pool_size)(conv_3)


  conv_4 = resblock(pool_3, 16*filter_num)
  pool_4 = MaxPooling2D(pool_size)(conv_4)


  conv_5 = resblock(pool_4, 32*filter_num)


  up_1 = upsample_concat_with_attention_block(conv_5, conv_4,16*filter_num)
  up_1 = resblock(up_1, 16*filter_num)


  up_2 = upsample_concat_with_attention_block(up_1, conv_3,8*filter_num)
  up_2 = resblock(up_2, 8*filter_num)


  up_3 = upsample_concat_with_attention_block(up_2, conv_2,4*filter_num)
  up_3 = resblock(up_3, 4*filter_num)


  up_4 = upsample_concat_with_attention_block(up_3, conv_1,2*filter_num)
  up_4 = resblock(up_4, 2*filter_num)


  out = Conv2D(n_classes, (1,1), kernel_initializer='he_normal', padding='same', activation='sigmoid')(up_4)
  Res_Unet_with_att_model=Model(Xinput, out)
  return Res_Unet_with_att_model

In [ ]:
inputshape = (256,256,3)
Res_Unet_model=Res_Unet_with_att_model(inputshape,filter_num=16)

In [ ]:
Res_Unet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, ids , mask, image_dir = './', batch_size = 16, img_h = 256, img_w = 256, shuffle = True):
    self.ids = ids
    self.mask = mask
    self.image_dir = image_dir
    self.batch_size = batch_size
    self.img_h = img_h
    self.img_w = img_w
    self.shuffle = shuffle
    self.on_epoch_end()
  def __len__(self):
    return int(np.floor(len(self.ids)) / self.batch_size)

  def __getitem__(self, index):
    indexes = self.indexes[index* self.batch_size : (index+1) * self.batch_size]
    list_ids = [self.ids[i] for i in indexes]
    list_mask = [self.mask[i] for i in indexes]
    X, y = self.__data_generation(list_ids, list_mask)
    return X, y

  def on_epoch_end(self):    
    self.indexes = np.arange(len(self.ids))

    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_ids, list_mask):

    X = np.empty((self.batch_size, self.img_h, self.img_w, 3))
    y = np.empty((self.batch_size, self.img_h, self.img_w, 3))

    for i in range(len(list_ids)):
      img_path = str(list_ids[i])
      mask_path = str(list_mask[i])
      img = io.imread(img_path)
      mask = io.imread(mask_path)
      img = cv2.resize(img,(self.img_h,self.img_w))
      img = np.array(img, dtype = np.float64)
      mask = cv2.resize(mask,(self.img_h,self.img_w))
      mask = np.array(mask, dtype = np.float64)
      img -= img.mean()
      img /= img.std()
      mask -= mask.mean()
      mask /= mask.std()
      X[i,] = img
      y[i,] = mask
    y = (y > 0).astype(int)
    return X, y

train_data = DataGenerator(train_ids, train_mask)
val_data = DataGenerator(val_ids, val_mask)

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=20
                             )
checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/CT_res_att_unet/CT_res_att_unet.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.0001,
                              factor=0.2
                             )
filename='/content/drive/MyDrive/CT_res_att_unet/history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [ ]:
adam = tf.keras.optimizers.Adam(lr = 0.02, epsilon = 0.1)
Res_Unet_model.compile(optimizer = adam, 
                  loss = focaltversky, 
                  metrics = [Recall(), Precision(), tversky, MeanIoU(num_classes=3)]
                 )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
Res_Unet_model.load_weights("/content/drive/MyDrive/CT_res_att_unet/CT_res_att_unet.h5")

In [ ]:
Res_Unet_model.fit(train_data, 
                  epochs = 200,
                  initial_epoch=33,
                  validation_data = val_data,
                  callbacks = [checkpointer, earlystopping, reduce_lr,history_logger]
                 )

Epoch 34/200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


391/391 [==============================] - ETA: 0s - loss: 0.9007 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - tversky: 0.1266 - mean_io_u_1: 0.4997
Epoch 34: val_loss improved from inf to 0.91124, saving model to /content/drive/MyDrive/CT_res_att_unet/CT_res_att_unet.h5
391/391 [==============================] - 375s 946ms/step - loss: 0.9007 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - tversky: 0.1266 - mean_io_u_1: 0.4997 - val_loss: 0.9112 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_tversky: 0.1156 - val_mean_io_u_1: 0.4997 - lr: 0.0200
Epoch 35/200
391/391 [==============================] - ETA: 0s - loss: 0.9119 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - tversky: 0.1133 - mean_io_u_1: 0.4997
Epoch 35: val_loss did not improve from 0.91124
391/391 [==============================] - 369s 944ms/step - loss: 0.9119 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - tversky: 0.1133 - mean_io_u_1: 0.4997 - val_loss: 0.9213 - val_recall_1: 0.0000e+